<a href="https://colab.research.google.com/github/osullik/ic23050/blob/main/DataDictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
from google.colab import drive
import pandas as pd
import json
from collections import defaultdict

In [143]:
DD = pd.ExcelFile("https://github.com/osullik/ic23050/blob/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/DataDictionary_Washington%20Fatal%20Crash%20Files.xlsx?raw=true")

In [144]:
for sheet in DD.sheet_names:
  print(sheet)
  


ReadMe
CRASH Data Files
PERSON Data Files
Tables1
Tables2
Tables3
Tables4
Tables5
Tables6
Tables7
Tables8
Tables9


In [249]:
#Seperate out each sheet in the data dictionary 
ReadMe = pd.read_excel(DD, 'ReadMe')
CrashDataFiles = pd.read_excel(DD, 'CRASH Data Files')
PersonDataFiles = pd.read_excel(DD, 'PERSON Data Files')
Tables1 = pd.read_excel(DD, 'Tables1')
Tables2 = pd.read_excel(DD, 'Tables2')
Tables3 = pd.read_excel(DD, 'Tables3')
Tables4 = pd.read_excel(DD, 'Tables4')
Tables5 = pd.read_excel(DD, 'Tables5')
Tables6 = pd.read_excel(DD, 'Tables6')
Tables7 = pd.read_excel(DD, 'Tables7')
Tables8 = pd.read_excel(DD, 'Tables8')
Tables9 = pd.read_excel(DD, 'Tables9')

In [321]:
'''
getKVPairs is a function to extract KV Pairs from dirty excel data
  Input Args:
    desc: a String "description" of the values we want to extract. Should 
      include target KV pairs in the form KEY=VALUE where KEY is a number and 
      VALUE are non-numeric substrings. 
  Method: 
    Break the string into characters
    Accumulate numeric values until an "=" is seen. The "=" is an assignment  
      we infer the numbers prior are the key and this is appended to keyList
    Accumulate characters until either EOF or another numerical value is found
      we infer all strings between keys are their associated values. 
    
    Before appending a value to the definition list, remove newline and 
      truncate spaces

    Zip the KeyList and DefinitionList together, and return

  Returns:
    A list of Tuples in the form [(Key, Value)]

'''
def getKVPairs(desc):
                                                #Init Vars
  number = ""
  definition = ""
  keyList = []
  definitionList = []
  keyFound = False
                                                #Filter out non-strings
  if type(desc) == str:
    for char in desc:
      
      if char.isnumeric() == True:              #Build the Keys
        if keyFound == True:
          definition = definition.strip("\n")
          definition = definition.strip()
          definitionList.append(definition)
          definition = ""
          keyFound = False
          number += char

        else:                                   #Build the Values
          number += char

      elif char == ("="):                       #Commit the Key
        keyList.append(number)
        number = ""
        keyFound = True

      else:                                     #Build the value
        definition += char 
    else:
      pass
                                                #Add the last Definition
    definition = definition.strip("\n")
    definition = definition.strip()
    definitionList.append(definition)

  toReturn = list(zip(keyList,definitionList))  #Create a list of (K,V) Tuples

  return(toReturn)    

In [335]:
#Parse the CrashDataFile into JSON
dataDict = defaultdict(dict)                    #Build the Data Dictionary

for idx, series in CrashDataFiles.iterrows():   #Step through each var in turn
  var = series['VARIABLE']
  typ = series['TYPE']
  length = series['LENGTH']
  format = series['FORMAT']
  label = series['LABEL']
  desc = series['DESCRIPTION/COMMENT']

                                                #Populate the Data Dictionary
  dataDict[var]["TYPE"]     = typ
  dataDict[var]["LENGTH"]   = length 
  dataDict[var]["FORMAT"]   = format 
  dataDict[var]["LABEL"]    = label 

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA

  dataDict[var]["DATA"]    = dict(kvPairList)

  


In [334]:
json_CrashDataFile = json.dumps(dataDict, indent = 4) 
print(json_CrashDataFile)

{
    "year": {
        "TYPE": 1.0,
        "LENGTH": 8.0,
        "FORMAT": NaN,
        "LABEL": "Year",
        "DATA": {}
    },
    "par": {
        "TYPE": 2.0,
        "LENGTH": 7.0,
        "FORMAT": NaN,
        "LABEL": "Police Accident Report Number",
        "DATA": {}
    },
    "investjur": {
        "TYPE": 2.0,
        "LENGTH": 20.0,
        "FORMAT": NaN,
        "LABEL": "Investigating Agency",
        "DATA": {}
    },
    "repjur": {
        "TYPE": 1.0,
        "LENGTH": 8.0,
        "FORMAT": NaN,
        "LABEL": "Reporting Jurisdiction",
        "DATA": {}
    },
    "crash_dt": {
        "TYPE": 1.0,
        "LENGTH": 8.0,
        "FORMAT": "MMDDYY",
        "LABEL": "Crash Date",
        "DATA": {}
    },
    "crash_tm": {
        "TYPE": 1.0,
        "LENGTH": 8.0,
        "FORMAT": "TIME",
        "LABEL": "Crash Time",
        "DATA": {}
    },
    "accday": {
        "TYPE": 1.0,
        "LENGTH": 8.0,
        "FORMAT": NaN,
        "LABEL": "Crash Day",

In [337]:
#Parse the PersonDataFile into JSON
dataDict_PersonData = defaultdict(dict)                    #Build the Data Dictionary

print(PersonDataFiles.columns)

for idx, series in PersonDataFiles.iterrows():   #Step through each var in turn
  var = series['VARIABLE']
  typ = series['TYPE']
  length = series['LENGTH']
  format = series['FORMAT']
  label = series['LABEL']
  desc = series['DESCRIPTION/COMMENT']

                                                #Populate the Data Dictionary
  dataDict_PersonData[var]["TYPE"]     = typ
  dataDict_PersonData[var]["LENGTH"]   = length 
  dataDict_PersonData[var]["FORMAT"]   = format 
  dataDict_PersonData[var]["LABEL"]    = label 

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA

  dataDict_PersonData[var]["DATA"]    = dict(kvPairList)

Index(['Unnamed: 0', 'VARIABLE', 'TYPE', 'LENGTH', 'FORMAT', 'LABEL',
       'DESCRIPTION/COMMENT', 'Notes / SAS Code', 'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')


In [338]:
json_PersonDataFile = json.dumps(dataDict_PersonData, indent = 4) 
print(json_PersonDataFile)

{
    "year": {
        "TYPE": 1.0,
        "LENGTH": 8.0,
        "FORMAT": NaN,
        "LABEL": "Year",
        "DATA": {}
    },
    "par": {
        "TYPE": 2.0,
        "LENGTH": 7.0,
        "FORMAT": NaN,
        "LABEL": "Police Accident Report Number",
        "DATA": {}
    },
    "investjur": {
        "TYPE": 2.0,
        "LENGTH": 20.0,
        "FORMAT": NaN,
        "LABEL": "Investigating Agency",
        "DATA": {}
    },
    "crash_dt": {
        "TYPE": 1.0,
        "LENGTH": 8.0,
        "FORMAT": "MMDDYY",
        "LABEL": "Crash Date",
        "DATA": {}
    },
    "crash_tm": {
        "TYPE": 1.0,
        "LENGTH": 8.0,
        "FORMAT": "TIME",
        "LABEL": "Crash Time",
        "DATA": {}
    },
    "accday": {
        "TYPE": 1.0,
        "LENGTH": 8.0,
        "FORMAT": NaN,
        "LABEL": "Crash Day",
        "DATA": {}
    },
    "accmon": {
        "TYPE": 1.0,
        "LENGTH": 8.0,
        "FORMAT": NaN,
        "LABEL": "Crash Month",
        "D

In [341]:
dataDict_HarmfulEvent = defaultdict(dict)                    #Build the Data Dictionary

#print(Tables1.columns)

#print(Tables1['COUNTY NAME'])

Tables1

#for idx, series in PersonDataFiles.iterrows():   #Step through each var in turn
#  var = series['VARIABLE']
#  typ = series['TYPE']
#  length = series['LENGTH']
#  format = series['FORMAT']
#  label = series['LABEL']
#  desc = series['DESCRIPTION/COMMENT']

,COUNTY NAME,FIRST HARMFUL EVENT,Unnamed: 2,Unnamed: 3,SEATING POSITION
0,county_char,Non-Collision,NaN,NaN,0=Not a Motor Vehicle Occupant (2005-Later)
1,1=Adams,01=Rollover/Overturn,NaN,NaN,11=Front Seat – Left Side (Driver)
2,3=Asotin,02=Fire/Explosion,NaN,NaN,12=Front Seat – Middle
3,5=Benton,03=Immersion or Partial Immersion,NaN,NaN,13=Front Seat – Right Side
4,7=Chelan,04=Gas Inhalation,NaN,NaN,18=Front Seat – Other
...,...,...,...,...,...
56,29=Unknown Regulatory Sign,NaN,NaN,78 - RR-Other Passive Device,NaN
57,30=School Speed Limit,NaN,NaN,"79 - RR-Passive Device, Type Unknown",NaN
58,31=School Advance/Crossing,NaN,NaN,"80 - RR-Grade Crossing Control, Type Unknown",NaN
59,38=Other School Related,NaN,NaN,98 - Other,NaN
